<a href="https://colab.research.google.com/github/brilianputraa/deepiduav/blob/master/DEEPIDUAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# System Identification of UAV using Deep Learning Methods

## Dataset Preparation

In this experiment, the flight data of UAV (Quadcopter) were obtained from Parrot Mambo Fly drone.

The data acquisition was processed using [MATLAB Support Package for Parrot Drones](https://www.mathworks.com/help/supportpkg/parrotio/referencelist.html?type=function&listtype=cat&category=index&blocktype=all&capability=&s_tid=CRUX_lftnav), the flight data were collected directly when the drone was hovering. By utilizing the support package we can collect the UAV flight data such as the UAV attitude orientation, each motor speed, etcetera in real-time.

